# CSE 6240/CX4803 Web Search and Text Mining
## Homework 3: Centrality and Pagerank

This homework allows you to explore with centrality and pagerank algorithms on a real world social network.

In [1]:
!pip install networkx==2.6.3 scipy==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


Please run the following cell substituting your student and user names.

In [2]:
## Do not change or add any other libraries ## 
import networkx as nx
import numpy as np
from copy import deepcopy
from scipy.stats import pearsonr
import scipy.sparse as sp
import random

In [3]:
def author_honor_code (student_name, user_name):
  print (f'I, {student_name} ({user_name}), state that I performed the tasks in this assignment following the Georgia Tech honor code (https://osi.gatech.edu/content/honor-code).')

# print the honor code before submission (substitute your name and username)
author_honor_code (student_name='Yeojin Chang', user_name='ychang354')

I, Yeojin Chang (ychang354), state that I performed the tasks in this assignment following the Georgia Tech honor code (https://osi.gatech.edu/content/honor-code).


**Important NOTE: To remove any randomness, please make sure to pass the parameter `v0` (i.e., initial estimate for the eigenvector) to any call to scipy.linalg.eigsh as the all-ones vector of size (N,), where N is the number of nodes.**

# Data Loading: GitHub network [0.1 points]

Throughout this assignment, we will use a large social network from [GitHub](https://github.com). The following cell will download the network in your current directory. If it does not work, please download the zip file from [here](http://snap.stanford.edu/data/git_web_ml.zip) and unzip.

The GitHub network can be described as follows:

- A node in the network represents a developer account that is active.
- Two nodes are linked if they mutually follow each other.

There is other information related to the nodes in the data you just downloaded, but for the purpose of this assignment we will ignore it.

In [4]:
![ ! -d "git_web_ml" ] && wget http://snap.stanford.edu/data/git_web_ml.zip && unzip git_web_ml.zip


--2023-02-22 16:04:54--  http://snap.stanford.edu/data/git_web_ml.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2396031 (2.3M) [application/zip]
Saving to: ‘git_web_ml.zip’

git_web_ml.zip      100%[===================>]   2.28M  2.21MB/s    in 1.0s    

2023-02-22 16:04:55 (2.21 MB/s) - ‘git_web_ml.zip’ saved [2396031/2396031]

Archive:  git_web_ml.zip
   creating: git_web_ml/
  inflating: git_web_ml/musae_git_edges.csv  
  inflating: git_web_ml/musae_git_features.json  
  inflating: git_web_ml/musae_git_target.csv  
  inflating: git_web_ml/citing.txt   
  inflating: git_web_ml/README.txt   


In [5]:
def load_net (filename):
  """ Load the network from the file.
  Arguments
  ---------
  filename (str): The name of the file which 
  contains the edges.

  Returns
  -------
  networkx.Graph

  Steps:
  1. Split each line by comma.
  2. Add edges to the network.
  """
  net = nx.Graph()
  with open (filename) as fin:
    for i, line in enumerate (fin):
      if i > 0:
        ## Add code below [0.1 points] ##
        e = line.split(',')
        net.add_edge(e[0], e[1][:-1])
        #################################
  return net

In [6]:
## Do not change ## 
net = load_net ('git_web_ml/musae_git_edges.csv')

In [7]:
## Do not change. This should print True if your code is correct. ## 
print ((net.number_of_nodes() == 37700) and (net.number_of_edges() == 289003))

True


# Section 1: Centrality measures [0.4 points]

A common question in network analysis is to find the central nodes in the network. These nodes are likely to be more important and more influential. In this section, we'll first calculate the degree, betweenness, and eigenvector centrality for the GitHub network by calling `networkx` functions. 

The betweenness centrality calculation can be extremely slow. To speed it up, please use $100$ node samples to approximate the value instead of all the nodes. This can be achieved by the networkx function.

In [8]:
random.seed (42)
np.random.seed (42)
## Add code to calculate the different centrality measures [0.2 points] ##
degree_centrality = nx.degree_centrality(net)
betweenness_centrality = nx.betweenness_centrality(G = net, k = 100)
eigenvector_centrality = nx.eigenvector_centrality(net)
###########################################################################

In [9]:
# Do not change
def get_top_central_nodes (centrality_scores, top_k=10):
  """ Given the centrality scores dict, give the top_k
      nodes with the highest centrality
  """
  return np.array(list(sorted (centrality_scores.keys(), key=lambda x:centrality_scores[x], reverse=True))[:top_k])

top5_deg = get_top_central_nodes (degree_centrality, top_k=5)
top5_bw = get_top_central_nodes (betweenness_centrality, top_k=5)
top5_eig = get_top_central_nodes (eigenvector_centrality, top_k=5)

In [10]:
# Do not change. This should print True if your code is correct.
print (np.all(top5_deg == top5_eig) and (np.all(top5_bw[[1,0,3,2,4]] == top5_deg)))

True


Now calculate the Pearson's correlation between these centrality metrics using `pearsonr` function from `scipy.stats`

Note: Make sure that the input to the functions are the centrality values for the same sequence of nodes.

In [11]:
## Add code to calculate the pearson correlation between the different centrality measures [0.1 points] ##
corr_deg_bw = pearsonr(list(degree_centrality.values()), list(betweenness_centrality.values()))[0]
corr_deg_eig = pearsonr(list(degree_centrality.values()), list(eigenvector_centrality.values()))[0]
corr_bw_eig = pearsonr(list(betweenness_centrality.values()), list(eigenvector_centrality.values()))[0]
##########################################################################################################

In [12]:
# Do not change. This should print True if your code above is correct.
print (np.all (np.abs([corr_deg_bw - 0.873, corr_bw_eig - 0.643, corr_deg_eig - 0.858]) <= 5e-3))

True


Explain why certain centrality measures are more correlated than others ? [0.1 points]

> In this graph, we can observe that the correlation between degree centrality and betweennesss centrality and the correlation between degree centrality and eignvector centrality is significantly higher than that of betweenness centrality and eignvector centrality. This is because the calculations for betweenness centrality and eigenvector centrality somewhat depend on the degrees of a node. For instance, if a node has a lot of edges leading in/out of the node (degree centrality), it is more likely to see more information when the shortest path is taken (betweenness centrality). However, betweenness and eigenvector centrality don't necessarily have this correlation.

# Section 2: HITS and PageRank  [6.5 points]

In this section, we'll then move to link analysis and implement vectorized (hence, scalable) versions of the HITS and PageRank algorithm to find the more important nodes in the network. Finally, we'll compare our PageRank scores with `networkx` Pagerank scores and scores from the HITS algorithm.




To get started, we will first convert the network into a directed network
(This is not strictly necessary, but HITS and PageRank work well on a directed network). We will simply convert the edges to directed edges by making them point to nodes that have a lower id.


In [13]:
# Do not change. Function to convert to directed graph
def convert_to_directed (net):
  G = nx.DiGraph ()
  for source, target in net.edges ():
    if int (source) > int (target):
      G.add_edge (source, target)
    else:
      G.add_edge (target, source)
  
  return G

In [14]:
# Get directed graph. Do not change.
di_net = convert_to_directed (net)
A = nx.linalg.graphmatrix.adjacency_matrix (di_net)
nodeid2rownum = {node: i for i, node in enumerate (di_net.nodes())}
rownum2nodeid = {i: node for i, node in enumerate (di_net.nodes())}

## HITS algorithm [3.0 points]

We will implement HITS in 3 different ways: (i) Matrix Multiplication Iterative method, (ii) NetworkX functions, and (iii) Eigenvector method

### Matrix Multiplication Iterative HITS algorithm [2.5 points]



You will implement the matrix formulation of HITS algorithm as given by the following iterative algorithm.

$$
\vec{h}^{(0)} = 1, \\
\vec{a}^{(0)} = 1, \\ 
\\
\vec{h}^{(t+1)} = c_1 A \vec{a}^{(t)}, \\
\vec{a}^{(t+1)} = c_2 A^T \vec{h}^{(t)},
$$

where $\vec{a}^{(t)}, \vec{h}^{(t)}$ are the authority and hub scores of the nodes at time $t$; $A$ is the adjacency matrix of the graph. And $c_1, c_2$ are some constants. 


After each iteration, it is important to normalize the authority and hub scores over all the nodes to make the algorithm converge by keeping the scores bounded within $1$. In particular, we do the following:

$$
\vec{a}^{(t)} = \vec{a}^{(t)}/\lVert \vec{a}^{(t)} \rVert \\
\vec{h}^{(t)} = \vec{h}^{(t)}/\lVert \vec{h}^{(t)} \rVert
$$

Note that you don't need to explicitly set $c_1, c_2$ as the normalization will take care of it.

Note that we are asking you to use matrix multiplication methods, instead of summing the neighbors' hub or authority values. The latter is slow. Matrix multiplication methods are much faster. 


In [15]:
def my_hits (G, A, num_iters=50, tol=1e-3):
  """ Implement the matrix multiplication-based iterative HITS algorithm

  Arguments
  ---------
  G (networkx.DiGraph): The directed network for which pagerank is to be calculated.
  A (scipy.sparse.matrix): The adjacency matrix.
  num_iters (int): The max number of iterations.
  tol (float): The tolerance limit. Exit the algorithm if num_iters is reached or scores from 
               current iteration differ from the previous iteration only upto tolerance specified.
  Returns
  -------
  hub_scores, auth_scores: Two dictionaries storing hub and authority scores keyed by the nodes in the graph.

  ###########################################
  ## Steps:
  # 1. Initialize the hub and auth scores. Each vector should be initialized for each node with a score of 1. 
  # 2. For num_iters iterations, iteratively update the auth and hub scores according to the above equation. 
  # 3. In each iteration, calculate new hub and authority score vectors from current authority and hub score vectors, respectively. 
  #    Use the appropriate matrix multiplication operations.
  # 4. In each iteration, normalize the new vectors. Use the np.linalg.norm function appropriately. 
  # 5. In each iteration, check for exit condition. Note that the exit condition should ensure tolerance limit is satisfied for both hub and authority scores.
  #    -> Calculation of tolerance should be done by computing the mean sqaured error (MSE) between the previous and current value.
  # 6. After all the iterations are completed, create the hub_scores and auth_scores dictionaries. Each dictionaries has node id as key and hub or authority score as value. 
  
  ## Notes for faster implementation:
  # - Implement the vectorized versions of the above equations so that the code can efficiently run for 100 iterations. 
  #   Summing up neighbors' scores is slow and should not be used. 
  # - For faster implementation, you can compute the new and old authority and hub scores using matrix multiplications. Note that this will be a multiplication between a 
  #   scipy sparse matrix and a numpy array vector. Refer https://docs.scipy.org/doc/scipy/reference/sparse.html#matrix-vector-product or 
  #   you may directly use the @ (or matrix multiplication) operation. 
  # - Make sure that normalization is also done in a vectorized manner. 
  """
  hub_scores, auth_scores = None, None
  # Add code below
  
  # step 1: initialize
  num_nodes = di_net.number_of_nodes()
  hub_scores = np.full(num_nodes, 1)
  auth_scores = np.full(num_nodes, 1)

  # step 2: iterate
  for i in range(num_iters):
    # step 3: update
    hub_t = hub_scores
    auth_t = auth_scores
    hub_scores = A@auth_t
    auth_scores = A@hub_t

    # step 4: normalize
    hub_scores = hub_scores/np.linalg.norm(hub_scores)
    auth_scores = auth_scores/np.linalg.norm(auth_scores)

    # step 5: exit condition
    if sum(np.square(hub_scores - hub_t)) < tol:
      hub_scores = {rownum2nodeid[i]: hub_scores[i] for i, node in enumerate(G.nodes())}
      auth_scores = {rownum2nodeid[i]: auth_scores[i] for i, node in enumerate(G.nodes())}
      return hub_scores, auth_scores

    if sum(np.square(auth_scores - auth_t)) < tol:
      hub_scores = {rownum2nodeid[i]: hub_scores[i] for i, node in enumerate(G.nodes())}
      auth_scores = {rownum2nodeid[i]: auth_scores[i] for i, node in enumerate(G.nodes())}
      return hub_scores, auth_scores

    

  hub_scores = {rownum2nodeid[i]: hub_scores[i] for i, node in enumerate(G.nodes())}
  auth_scores = {rownum2nodeid[i]: auth_scores[i] for i, node in enumerate(G.nodes())}

  ####################################################

  return hub_scores, auth_scores

In [16]:
# %%time 
# Do not change. An efficient solution would be able to run this within one second. 
myhub_scores, myauth_scores = my_hits(di_net, A, num_iters=100, tol=1e-8)

### Calculating HITS with Networkx [0.1 points]
Now, let's check how your `my_hits` perform as compared to the networkx's implementation of the iterative algorithm. 


Calculate the hubs and authority scores using built-in `networkx` hits function. You're allowed to use `nx` functions.  Use default parameters for num_iters and tol (which are same as for the call to my_hits above).

In [17]:
## Add code below to get hub and authority scores from networkx [0.1 points] ##
# This should return hub and authority scores as key-value pairs in a dictionary where keys are nodes.
nxhub_scores, nxauth_scores = nx.hits(G=di_net, max_iter=100, tol=1e-8)
###############################################################################

In [18]:
# Do not change. This should print True if your code above is correct.
hub_diff = np.array([np.abs(nxhub_scores[x] - myhub_scores[x]) for x in myhub_scores])
auth_diff = np.array([np.abs(nxauth_scores[x] - myauth_scores[x]) for x in myauth_scores])

print (auth_diff.mean() <= 5e-3 and hub_diff.mean() <= 1e-3)

True


Thus, the difference is small and implementation is good.

### Calculating HITS with Eigenvectors [0.4 points] 

We know that the iteration in my_hits converges as $t \rightarrow \infty$. 

Now, let us calculate how close the calculated iterative values are to eigenvector-based HITS values. Recall that eigenvector-based HITS values are the final values at convergence.  

You will use scipy.linalg libraries to calculate eigenvectors of matrices. Then, you will measure how close the values obtained by `my_hits` function are to the eigenvector methods. 

In [19]:
## Write code to find the authority and hub scores by finding the appropriate eigenvectors ##
# Notes:
#  - Calculate the principal eigenvector of the appropriate matrices to calculate the authority and hub scores respectively. 
#    Use sp.linalg.eigsh function to calculate eigenvectors. Please make sure to pass the parameter `v0` 
#    (initial estimate for the eigenvector) as the all-ones vector of size (N,), where N is the number of nodes. 
#  - Again, use the @ (or matrix multiplication) operation for faster computations.
#  - Note that true_auth_scores and true_hub_scores should be an array of dimension (N,) and type float (where N is the number of nodes)
true_auth_scores, true_hub_scores = None, None
# Add code below:
# 
n = A.shape[0]
ata = sp.linalg.eigsh(A = (A.T@A).astype(float), v0=np.full(n, float(1.0)))
true_auth_scores = ata[1][:, 0]

aat = sp.linalg.eigsh(A = (A@A.T).astype(float), v0=np.full(n, float(1.0)))
true_hub_scores = aat[1][:, 0]

###############################################################################

In [20]:
# Do not change. This should print True if your code above is correct.
hvec = np.array([myhub_scores[rownum2nodeid[i]] for i in range(A.shape[0])])
avec = np.array([myauth_scores[rownum2nodeid[i]] for i in range(A.shape[0])])

print (np.abs(true_auth_scores - avec).mean() <= 6e-3) and (np.abs(true_hub_scores - hvec).mean() <= 2e-3)

True


## PageRank algorithm [3.2 points]

We will implement PageRank in 3 different ways: (i) Matrix Formulation of Iterative method, (ii) NetworkX functions, and (iii) Eigenvector method

### Matrix Formulation of Iterative PageRank algorithm [2.5 points]

Now, we will calculate Pagerank. The idea behind Pagerank is that a node is more important if other important nodes link to it. It's also the basic algorithm upon which Google ranked webpages for their search results. We'll implement a simplified iterative version of Pagerank.

You'll implement the matrix formulation of PageRank as the following iterative algorithm. 

$$
\vec{r}^{(0)} = (1/N, 1/N, \cdots, 1/N)^T \\
\vec{r}^{(t+1)} = \frac{1-p}{N} + p M \vec{r}^{(t)}, 
$$

where $\vec{r}^{(t)}$ is the PageRank scores of all the nodes at iteration $t$; $N$ is the total number of nodes in the network; $p$ is the transportation probability; $M$ is the stochastic random matrix defined as 

$$ M_{ij} = \begin{cases} 1/d^{out}_j &; (j, i) \text{ is an edge} \\ 0 &; \text{otherwise} \end{cases},$$

where $d^{out}_j$ is the out-degree of node $j$.

In [21]:
def get_stochastic_matrix (A):
  """
  Find the stochastic matrix M from the adjacency matrix A, for the pagerank score calculation. Use the above definition.
  Here, we list two possible ways of implementation and you are welcome to adopt any. 
  STEPS:
  # 1. Use A.nonzero() to obtain nonzero row and column indices of the adjacency matrix A. 
  # 2. Then use row, col to obtain the non-zero indices of M.
  # 3. Create a data array that stores the corresponding out-degrees of nodes with the same length as row and col arrays. 
  #    Make sure that the indices for row, col match with the d^{out}_j as the definition above. 
  # 4. Use sp.csr_matrix ((data, (row, column))) [https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html] 
  #    to find M. Note that M should be of the same size as A (hint: use the shape parameter of the `csr_matrix` function). 
  This is worth 1.25 points out of 2.5 points.
  """
  M = None
  # Add code below
  #
  n = A.shape[0]
  row_nonzeros, col_nonzeros = A.nonzero()
  dat = [1/np.count_nonzero(col_nonzeros==i) for i in col_nonzeros]

  M = sp.csr_matrix((dat, (row_nonzeros, col_nonzeros)), shape=(n, n))
  ####################################################
  return M

In [22]:
def my_pagerank (G, A, p=0.85, num_iters=50, tol=1e-3):
  """"
  Calculate the pagerank scores for a given graph.

  Arguments
  ---------
  G (networkx.DiGraph): The directed network for which pagerank is to be calculated.
  A (scipy.sparse.matrix): The adjacency matrix for the given directed network.
  p(float): p is the probability that a link is followed in a random walk. 1-p is the restart probability.
  num_iters (int): The number of iterations to perform
  tol (float): The tolerance value to keep track; algorithm should exit if 
               either the number of iterations are exhausted or difference between
               the pagerank scores across two consecutive iterations is within tolerance.
  Returns
  -------
  pr_scores (dict): The pagerank scores as key-value pairs in a dictionary where keys are nodes.
  ###########################################
  ## Steps:
  # 1. Initialize the pr_scores for each node as 1/N, where N is the total number of nodes.
  # 2. For num_iters iterations, iteratively update the pr_scores according to the above equation using M.
  # 3. In each iteration, calculate the new page rank scores from the current page rank scores and a uniform vector 1/N for teleportation.
  #    Use the correct matrix operations to calculate the new page rank scores. 
  # 4. In each iteration, check for exit condition. Recall that the loop exists when change in pagerank scores in iterations is less than tolerance value. 
  # 5. After all the iterations are completed, create the pr_scores dictionaries. Each dictionaries has node id as key and page rank score as value. 
  
  ## Notes:
  # - Implement the vectorized versions of the above equations so that the code can efficiently run for 
  #   100 iterations. Summing up neighbors' scores would be slow and should not be used. 
  # - Do not forget to check the tolerance condition at every step.
  # - Make sure that the returned scores are dictionaries storing pagerank scores keyed by the nodes. 
  """

  pr_scores = None
  M = get_stochastic_matrix (A)
  # Add code below
  # 
  # step 1
  N = G.number_of_nodes()
  pr_scores = np.full(N, 1/N)

  # step 2
  for i in range(num_iters):
    old_pr = pr_scores
    mcalc = M@old_pr
    pr_scores = (1 - p)/N + p*mcalc

    if sum(abs(pr_scores - old_pr)) < tol:
      pr_scores = {rownum2nodeid[i]: pr_scores[i] for i, node in enumerate(G.nodes())}
      return pr_scores

  pr_scores = {rownum2nodeid[i]: pr_scores[i] for i, node in enumerate(G.nodes())}

  ####################################################
  
  return pr_scores

In [23]:
# Do not change. An efficient solution should run within half a second.
mypr_scores = my_pagerank (di_net, A, p=0.85, num_iters=100, tol=1e-6)

### Calculating PageRank with Networkx [0.1 points]
Now, let's check how your `my_pagerank` perform as compared to the networkx's implementation of the iterative algorithm. 

Calculate the pagerank scores using the built-in `networkx` pagerank function. You're allowed to use `networkx` functions. Use default parameters for p, num_iters and tol (which are same as for the call to my_pagerank above).

In [24]:
## Add code below to get pageranks scores from networkx ##
# This should return pagerank scores as key-value pairs in a dictionary where keys are nodes. Use the correct networkx function. 
nxpr_scores = nx.pagerank(di_net, alpha=0.85)
#######################################################################

In [25]:
# Do not change. This should print True if your code above is correct.
pr_diff = np.array([np.abs(nxpr_scores[x] - mypr_scores[x]) for x in mypr_scores])

print (pr_diff.mean() <= 5e-5) 

True


Thus, the difference is very small and your implementation is good. 

### Calculating PageRank with Eigenvectors [0.6 points] 

We know that the above iteration converges as $t \rightarrow \infty$. Now, we will compare how close the scores are to the eigenvector-based PageRank values. Refer to the slides to calculate PageRank using the Eigenvector computation. Use linalg libraries of scipy/numpy. Then, compare values obtained by `my_pagerank` function to these eigenvector values.

**Pagerank convergence without random teleports**: First, let us assume p = 1 and find the pagerank. This simplifies the matrix you need to use for computation.

In [26]:
## Write code to find pagerank scores for p=1 that will be reached as number of iterations approaches infinity ##
# STEPS:
#  - Create the sparse stochastic matrix M. 
#  - Find its principal eigenvector (use sp.linalg.eigsh). Please make sure to pass the parameter `v0` 
#    (initial estimate for the eigenvector) as the all-ones vector of size (N,), where N is the number of nodes. 
# NOTES:
#  - Note that since p = 1, there is no teleportation. So, use the approriate matrix for the eigenvector computation.  
#  - You should use get_stochastic_matrix. 
#  - Note that true_pr_scores1 should be an array of dimension (N,) and type float (where N is the number of nodes)
true_pr_scores1 = None
# Add code below
# 
M = get_stochastic_matrix(A)
eig = sp.linalg.eigsh(A = M.astype(float), v0=np.full(n, float(1.0)))
true_pr_scores1 = eig[1][:, 0]

###############################################################################

In [27]:
# Compare the pagerank at p=1 with corresponding fixed point.
# Do not change. This should print True if your code above is correct.
mypr_scores1 = my_pagerank (di_net, A, p=1.0, num_iters=100, tol=1e-6) # Note that p is set to 1.0 here
prvec1 = np.array([mypr_scores1[rownum2nodeid[i]] for i in range(A.shape[0])])
print (np.abs(true_pr_scores1 - prvec1).mean() <= 5e-3)

True


**Pagerank convergence with random teleports**: Suppose you have a machine with RAM of 4 GB. What issues will you face when calculating pagerank scores (for an arbitrary $p < 1$) using the eigenvector method and why? [0.2 points]

The random transportations in PageRank will cause the stochastic matrix to be not sparse, as opposed to when p=1 and there is no random transportation. Since the eigenvalue decomposition of the stochastic matrix of dimensions $n \times n$ has a complexity of $O(n^3)$ where n is the number of nodes, it would be challenging for 4 GB of RAM to perform.

## PageRank vs HITS [0.3 points]
Now, let's compare the obtained pagerank and HITS scores. You only need to answer questions based on the following empirical observation.

In [28]:
# Do not change. This should print True if your code above is correct.
histdiff = np.histogram(np.abs([myhub_scores[x] - mypr_scores[x] for x in myhub_scores]))
print (histdiff[0][0]/len(myhub_scores) > 0.99 and histdiff[1][0] <= 1e-6)


True


Why are 99% of the pagerank scores similar to HITS scores?  [0.1 points]

> 99% of the pagerank score are similar to the HITS scores because most of the nodes have a very small score that is close to 0. This is because out of over 37K nodes, the chances of a random walker visiting one particular node is very small, and the hub/authority score calculations yield a very small score as well.

Why then may we prefer one over the other? Give two cases each when we may prefer HITS over Pagerank and vice-versa. [0.2 points]




> We would prefer HITS over Pagerank if we wanted users to be able to inflate the hub and authority scores easily, since in HITS if a page has a link to all other pages, it has a very high hub score, and any page linked to this page would have a high authority score. We would also prefer HITS if we are focusing purely on the structure of the web graph, and not if their textual contents are necessarily relevant, since it's a link-based algorithm.
On the other hand, we would prefer Pagerank over HITS if we wanted to add a level of personalization. By restricting the teleport set of nodes, Personalized Pagerank allows for the generation of recommendations in real time. We would also prefer Pagerank if we wanted a fairer system where a lay user can't just game the system, like HITS. This is because Pagerank calculates the likelihood of a user landing on a page through random walks.

# Section 3: Personalized PageRank [1.0 point]

Personalized Pagerank is calculated by changing the teleportation set to include only a set of nodes instead of all the nodes. Let us assume a set $S$ of teleporting nodes. Then, with a probability of (1-p), the nodes may teleport to any node in $S$ with probability $1/|S|$. In other words, we get the updated stochastic matrix to find PPR as

$$ M'_{ij} = \begin{cases} p M_{ij} + (1-p)/|S| & ;i \in S \\ p M_{ij} & ;\text{otherwise}  \end{cases}, $$

where $M$ is the stochastic random matrix of pagerank. Then, the iteration for PPR scores $\vec{r'}$ is given by $\vec{r'}^{(i+1)} \gets M' \vec{r'}^{(i)}$ in the $i$ th iteration. 

In this section, we will study how this "personalization" allows us to find the stationary distribution of the page rank scores for any p efficiently. 

We will find the fixed point of pagerank for $p=0.85$ when $S$ is nodes with top 10 degree centrality.

In [29]:
from networkx.readwrite.sparse6 import n_to_data
# Add code to find personalized pagerank scores with the teleport node set (i.e., S) as top 10 degree central nodes
# You are supposed to use the above code to obtain the 10 top-degree nodes and store it in the variable top10_deg.
# Assume probability of teleportation (i.e., 1-p) to be 0.15, which means p = 0.85
# This question is worth 0.8 points.
## STEPS:
#     1. Find the top 10 degree central nodes using `get_top_central_nodes` and store it in top10_deg.
#     2. Obtain the stochastic matrix M from adjacency matrix A using your previously-defined `get_stochastic_matrix`. 
#     3. Create a "teleporting" sparse matrix T with its (ij)th entry as 1/10 for all i in top10_deg and 0 otherwise. You may use sp.csr_matrix. 
#     4. Calculate M' using M and T. 
#     5. Return PPR scores for top10_deg, which would be the principal eigenvector of M' (use sp.linalg.eigsh). 
#        Please make sure to pass the parameter `v0` (initial estimate for the eigenvector) as the all-ones vector of 
#        size (N,), where N is the number of nodes. 
## NOTES: 
#  - Note that M is a sparse matrix as defined above.
#  - M' is also sparse as it only adds |S|*|V| more entries in the matrix
#  - Note that true_pprDeg_scores should be an array of dimension (N,) and type float (where N is the number of nodes)

p = 0.85
top10_deg = get_top_central_nodes(centrality_scores=degree_centrality, top_k=10)
true_pprDeg_scores = None
# Add code below:
M = get_stochastic_matrix(A)
n = A.shape[0]
dat = np.full(10 * n, 1/10)

rows_ = [[i]*n for i in top10_deg]
flat_row = sum(rows_, [])
cols = list(top10_deg) * n

T = sp.csr_matrix((dat, (flat_row, cols)), shape=(n,n))
M_prime = p * M + (1-p)*T

true_pprDeg_scores = sp.linalg.eigsh(A = M_prime.astype(float), v0=np.full(n, float(1.0)))[1][:, 0]

# 
###############################################################################

Let's compare these scores with the fixed point page rank scores with no random teleportation. Thus, the only difference between `true_pprDeg_scores` and `true_pr_scores1` is that the former randomly teleports to top-degree nodes. 

In [30]:
# Let's compare the pagerank at p=1 with PPR for top-10 degree teleportation nodes.
# Do not change. This should print True if your code above is correct.
ppr_diff = np.abs(true_pprDeg_scores - true_pr_scores1)
maxdiff10 = [rownum2nodeid[x] for x in sorted(range(len(true_pprDeg_scores)), key=lambda x: ppr_diff[x], reverse=True)[:10]]
print (len(set(top10_deg).difference(maxdiff10)) <= 3) 

True


Why is it expected that page rank scores are likely to differ the most for the nodes in set $S$ only ? For which other nodes do the page rank scores change by adding teleportation to top degree nodes? [0.2 points]



> The page rank scores are likely to differ the most for the nodes in Set S only because the nodes in S are given higher "priority" in personalized page rank, since 1-p proportion of the time, the random walker with teleport to one of these 10 nodes. The page rank scores would also change for nodes that are "close to" these top 10 nodes, since the likelihood that the random walker will continue its journey after teleportation onto one of these neighboring nodes will increase.